In [2]:
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QListWidget, QProgressBar, QTextEdit, QTabWidget, QTableWidget, QTableWidgetItem,
    QHeaderView, QMessageBox , QAction , QListWidgetItem
)
from PyQt5.QtCore import QThread, pyqtSignal, Qt
from PyQt5.QtGui import QPixmap, QImage, QFont ,QIcon

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import cv2

from datetime import datetime
import csv
import sys


import matplotlib.pyplot as plt
import numpy as np

In [4]:
from tensorflow.keras.models import load_model

model = load_model('projet_ML(PNEUMONIA).keras')

In [6]:
class PredictionAnalyzer(QThread):
    analysis_complete = pyqtSignal(dict)

    def __init__(self, model, image_path):
        super().__init__()
        self.model = model
        self.image_path = image_path
        self.categories = ['NORMAL', 'PNEUMONIA']

    def run(self):
        try:
            # Préparation de l'image
            img = cv2.imread(self.image_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_resized = cv2.resize(img_rgb, (224, 224))
            img_array = np.expand_dims(img_resized, axis=0)

            # Prédiction détaillée
            prediction = self.model.predict(img_array)
            predicted_label_index = np.argmax(prediction, axis=-1)[0]
            predicted_label = self.categories[predicted_label_index]
            confidence = prediction[0][predicted_label_index] * 100

            # Analyse complémentaire
            detailed_analysis = {
                'date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'label': predicted_label,
                'confidence': confidence,
                'raw_prediction': prediction[0].tolist(),
                'image_path': self.image_path
            }

            self.analysis_complete.emit(detailed_analysis)
        except Exception as e:
            print(f"Erreur d'analyse : {e}")
            self.analysis_complete.emit({})

class MedicalAIClassifier(QMainWindow):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.dark_mode = False  # Par défaut, mode clair
        self.diagnostic_history = []
        self.initUI()

    def initUI(self):
        # Fenêtre principale
        self.setWindowTitle("Assistant IA Médicale")
        self.setGeometry(100, 100,900, 800)
        self.setStyleSheet("""
            QMainWindow {
                background-color: #e6f2ff;
            }
            QPushButton {
                background-color: #2196F3;
                color: white;
                border-radius: 8px;
                padding: 12px;
                font-weight: bold;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #1976D2;
            }
            QLabel {
                font-size: 16px;
                color: #333;
            }
            QTextEdit {
                background-color: white;
                border: 2px solid #2196F3;
                border-radius: 10px;
                font-size: 16px;
                padding: 10px;
            }
        """)
        
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout()
        central_widget.setLayout(main_layout)

        # Titre principal
        title_label = QLabel('Diagnostic Médical par Intelligence Artificielle')
        title_label.setAlignment(Qt.AlignCenter)
        title_label.setFont(QFont('Arial', 20, QFont.Bold))
        main_layout.addWidget(title_label)
        
        # Chargement des icônes et styles
        self.load_icons()
        self.apply_theme()  # Thème clair ou sombre

        # Menu principal
        self.create_menu()

        # Création des onglets
        self.tabs = QTabWidget()
        self.setCentralWidget(self.tabs)

        # Onglet Diagnostic
        diagnostic_tab = QWidget()
        diagnostic_layout = QVBoxLayout(diagnostic_tab)

        # Zone pour afficher l'image
        self.image_label = QLabel("Aucune image sélectionnée")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setMinimumHeight(400)
        self.image_label.setStyleSheet("background-color: #ddffdd ; border: 2px dashed #cccccc;")
        diagnostic_layout.addWidget(self.image_label)

        # Boutons pour charger et analyser l'image
        btn_layout = QHBoxLayout()
        load_btn = QPushButton("Charger Image")
        load_btn.setIcon(QIcon(self.load_icon))
        load_btn.setToolTip("Chargez une radiographie au format PNG ou JPG")
        load_btn.clicked.connect(self.load_image)

        analyze_btn = QPushButton("Analyser Image")
        analyze_btn.setIcon(QIcon(self.analyze_icon))
        analyze_btn.setToolTip("Lancez l'analyse de l'image chargée")
        analyze_btn.clicked.connect(self.predict_image)

        btn_layout.addWidget(load_btn)
        btn_layout.addWidget(analyze_btn)
        diagnostic_layout.addLayout(btn_layout)

        # Résultats
        self.result_text = QTextEdit()
        self.result_text.setReadOnly(True)
        self.result_text.setStyleSheet("background-color: #ddffdd; border: 1px solid #ccc;")
        diagnostic_layout.addWidget(self.result_text)

        # Barre de progression
        self.progress_bar = QProgressBar()
        self.progress_bar.setValue(0)
        diagnostic_layout.addWidget(self.progress_bar)

        # Onglet Visualisation
        visualization_tab = QWidget()
        visualization_layout = QVBoxLayout(visualization_tab)
        self.figure_canvas = FigureCanvas(plt.Figure(figsize=(5, 4)))
        visualization_layout.addWidget(self.figure_canvas)

        # Onglet Historique
        history_tab = QWidget()
        history_layout = QVBoxLayout(history_tab)

        self.history_table = QTableWidget()
        self.history_table.setColumnCount(4)
        self.history_table.setHorizontalHeaderLabels(["Date", "Diagnostic", "Confiance (%)", "Chemin Image"])
        self.history_table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.history_table.setStyleSheet("background-color: #ddffdd;")
        history_layout.addWidget(self.history_table)

        # Boutons Historique
        history_buttons_layout = QHBoxLayout()
        clear_btn = QPushButton("Effacer Historique")
        clear_btn.clicked.connect(self.clear_history)

        export_btn = QPushButton("Exporter en CSV")
        export_btn.clicked.connect(self.export_history)

        history_buttons_layout.addWidget(clear_btn)
        history_buttons_layout.addWidget(export_btn)
        history_layout.addLayout(history_buttons_layout)

        # Ajout des onglets
        self.tabs.addTab(diagnostic_tab, "Diagnostic")
        self.tabs.addTab(visualization_tab, "Visualisation")
        self.tabs.addTab(history_tab, "Historique")

    def create_menu(self):
        # Barre de menu
        menu_bar = self.menuBar()

        # Menu Fichier
        file_menu = menu_bar.addMenu("Fichier")
        quit_action = QAction("Quitter", self)
        quit_action.setShortcut("Ctrl+Q")
        quit_action.triggered.connect(self.close)
        file_menu.addAction(quit_action)

        # Menu Affichage
        view_menu = menu_bar.addMenu("Affichage")
        toggle_theme_action = QAction("Mode Sombre", self)
        toggle_theme_action.triggered.connect(self.toggle_theme)
        view_menu.addAction(toggle_theme_action)

    def load_icons(self):
        # Chargement des icônes
        self.load_icon = "path/to/load_icon.png"
        self.analyze_icon = "path/to/analyze_icon.png"

    def load_image(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "Charger une radiographie", "", "Images (*.png *.jpg *.jpeg)")
        if file_name:
            self.current_image_path = file_name
            pixmap = QPixmap(file_name)
            scaled_pixmap = pixmap.scaled(600, 400, Qt.KeepAspectRatio, Qt.SmoothTransformation)
            self.image_label.setPixmap(scaled_pixmap)
            self.progress_bar.setValue(0)

    def predict_image(self):
        if not self.current_image_path:
            QMessageBox.warning(self, "Attention", "Veuillez charger une image avant d'analyser.")
            return

        self.progress_bar.setValue(0)
        self.prediction_thread = PredictionAnalyzer(self.model, self.current_image_path)
        self.prediction_thread.analysis_complete.connect(self.handle_prediction)
        self.prediction_thread.start()
        self.progress_bar.setValue(50)

    def handle_prediction(self, analysis):
        if not analysis:
            QMessageBox.warning(self, "Erreur", "L'analyse a échoué.")
            return

        label = analysis.get("label", "Inconnu")
        confidence = analysis.get("confidence", 0)
        result_message = f"Diagnostic: {label}\n"
        result_message += f"Probabilité: {confidence:.2f}%\n\n"
        if label == 'PNEUMONIA':
            result_message += "Recommandation: Consulter un médecin pour un examen approfondi."
            self.result_text.setStyleSheet("background-color: #ffdddd; color: #333;")
        else:
            result_message += "Résultat: Aucune anomalie détectée."
            self.result_text.setStyleSheet("background-color: #ddffdd; color: #333;")
        self.result_text.setText(result_message)
        self.update_visualization(analysis)
        self.update_history(analysis)
        self.progress_bar.setValue(100)


    
    def update_visualization(self, analysis):
        fig = self.figure_canvas.figure
        fig.clear()
        ax = fig.add_subplot(111)
        probabilities = analysis.get("raw_prediction", [])
        categories = ["NORMAL", "PNEUMONIA"]
        ax.pie(probabilities, labels=categories, autopct="%1.1f%%", startangle=90)
        self.figure_canvas.draw()

    def update_history(self, analysis):
        row_position = self.history_table.rowCount()
        self.history_table.insertRow(row_position)
        self.history_table.setItem(row_position, 0, QTableWidgetItem(analysis["date"]))
        self.history_table.setItem(row_position, 1, QTableWidgetItem(analysis["label"]))
        self.history_table.setItem(row_position, 2, QTableWidgetItem(f"{analysis['confidence']:.2f}%"))
        self.history_table.setItem(row_position, 3, QTableWidgetItem(analysis["image_path"]))

    def clear_history(self):
        reply = QMessageBox.question(self, "Effacer Historique", "Voulez-vous vraiment effacer tout l'historique ?", QMessageBox.Yes | QMessageBox.No)
        if reply == QMessageBox.Yes:
            self.history_table.setRowCount(0)
            self.diagnostic_history.clear()

    def export_history(self):
        file_name, _ = QFileDialog.getSaveFileName(self, "Exporter Historique", "", "CSV (*.csv)")
        if file_name:
            with open(file_name, "w", newline="") as file:
                writer = csv.writer(file)
                writer.writerow(["Date", "Diagnostic", "Confiance (%)", "Chemin Image"])
                for entry in self.diagnostic_history:
                    writer.writerow([entry["date"], entry["label"], entry["confidence"], entry["image_path"]])
            QMessageBox.information(self, "Exportation réussie", "L'historique a été exporté avec succès.")

    def toggle_theme(self):
        self.dark_mode = not self.dark_mode
        self.apply_theme()

    def apply_theme(self):
        if self.dark_mode:
            self.setStyleSheet("""
                QMainWindow {
                    background-color: #e6f2ff;
                }
                QPushButton {
                    background-color: #2196F3;
                    color: white;
                    border-radius: 8px;
                    padding: 12px;
                    font-weight: bold;
                    font-size: 16px;
                }
                QPushButton:hover {
                    background-color: #1976D2;
                }
                QLabel {
                    font-size: 16px;
                    color: #333;
                }
            QTextEdit {
                background-color: white;
                border: 2px solid #2196F3;
                border-radius: 10px;
                font-size: 16px;
                padding: 10px;
            }
            """)
        else:
            self.setStyleSheet("""
            QMainWindow {
                background-color: #e6f2ff;
            }
            QPushButton {
                background-color: #2196F3;
                color: white;
                border-radius: 8px;
                padding: 12px;
                font-weight: bold;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #1976D2;
            }
            QLabel {
                font-size: 16px;
                color: #333;
            }
            QTextEdit {
                background-color: white;
                border: 2px solid #2196F3;
                border-radius: 10px;
                font-size: 16px;
                padding: 10px;
            }
        """)



def launch_interface(model):
    app = QApplication(sys.argv)
    ex = MedicalAIClassifier(model)
    ex.show()
    sys.exit(app.exec_())

# Utilisation :
# launch_interface(votre_modele_entrainé)

In [8]:
launch_interface(model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step


SystemExit: 0

C:\Users\HOME\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
